In [42]:
#If running on colab
!rm -rf CelebA_Dataset
!git clone https://github.com/Nikhil4902/CelebA_Dataset.git

Cloning into 'CelebA_Dataset'...
remote: Enumerating objects: 16499, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 16499 (delta 0), reused 2 (delta 0), pack-reused 16492
Receiving objects: 100% (16499/16499), 101.13 MiB | 22.00 MiB/s, done.
Resolving deltas: 100% (412/412), done.


In [43]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalMaxPooling2D
from tensorflow.keras.optimizers import Adam
import numpy as np
from sklearn.metrics import confusion_matrix

In [44]:
image_size = (150, 150)
batch_size = 16
num_classes = 50
epochs = 100

In [45]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'CelebA_Dataset/Datasets/train_dataset',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

validate_generator = train_datagen.flow_from_directory(
    'CelebA_Dataset/Datasets/validation_dataset',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    'CelebA_Dataset/Datasets/test_dataset',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 1200 images belonging to 50 classes.
Found 150 images belonging to 50 classes.
Found 150 images belonging to 50 classes.


In [46]:
#Run only in case of training the model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(image_size[0], image_size[1], 3))

base_model.trainable = False

model = Sequential([
    base_model,
    Flatten(),
    Dense(2048, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [47]:
#Run only in case of training the model
model.fit(train_generator, epochs=epochs, validation_data=validate_generator)
model.save('face_identification_model.h5')

Epoch 1/100
75/75 [==============================] - 12s 147ms/step - loss: 4.1313 - accuracy: 0.0350 - val_loss: 3.6643 - val_accuracy: 0.0933
Epoch 2/100
75/75 [==============================] - 10s 133ms/step - loss: 3.7321 - accuracy: 0.0733 - val_loss: 3.4618 - val_accuracy: 0.1933
Epoch 3/100
75/75 [==============================] - 10s 135ms/step - loss: 3.5377 - accuracy: 0.1267 - val_loss: 3.3386 - val_accuracy: 0.1933
Epoch 4/100
75/75 [==============================] - 10s 134ms/step - loss: 3.3079 - accuracy: 0.1742 - val_loss: 3.1552 - val_accuracy: 0.2000
Epoch 5/100
75/75 [==============================] - 9s 117ms/step - loss: 3.1694 - accuracy: 0.2125 - val_loss: 3.0776 - val_accuracy: 0.2400
Epoch 6/100
75/75 [==============================] - 10s 127ms/step - loss: 3.0166 - accuracy: 0.2183 - val_loss: 2.8949 - val_accuracy: 0.3000
Epoch 7/100
75/75 [==============================] - 10s 134ms/step - loss: 2.9520 - accuracy: 0.2517 - val_loss: 2.8813 - val_accuracy: 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [48]:
model = tf.keras.models.load_model('face_identification_model.h5')
evaluation = model.evaluate(test_generator)

print("Test Loss:", evaluation[0])
print("Test Accuracy:", evaluation[1])

predictions = model.predict(test_generator)

predicted_classes = np.argmax(predictions, axis=1)
print(predicted_classes)

true_classes = test_generator.classes

conf_matrix = confusion_matrix(true_classes, predicted_classes)

print("Confusion Matrix:")
print(conf_matrix)

10/10 [==============================] - 1s 42ms/step - loss: 1.5311 - accuracy: 0.6467
Test Loss: 1.5311338901519775
Test Accuracy: 0.6466666460037231
10/10 [==============================] - 0s 34ms/step
[ 0  0  0  1 27  1 17  2 17  3 30  3 27 24 27  5  5  5  9 43  6  7  7 45
  8  8  8  0  9  9 10 10 11 11 11 11  9 24 12 46 13 41 44 14 14 15 47 15
 16 16 16 17 14 17 18 18 18 19 19 19 20 20 20 21 29 21 22 14 12 23 45 33
  4 24 24 31 45 25 26 17 26 47 27 27 28  8 18 29 40 29 30 25 30 31 31 31
 32 32 32  1  6 45 34 34 34  5 26 35 36 36 36 46 37 37 38 38 38 39 39 39
 40 40 40 17 41 22  5 42 35 43 43 43 44 44 35 34  7 45 31 46 43 29 26 47
 31 48 48 49 44 27]
Confusion Matrix:
[[3 0 0 ... 0 0 0]
 [0 2 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 2 0]
 [0 0 0 ... 0 0 1]]
